## Load data ##

In [22]:
import pickle
import pandas as pd
import numpy as np

def loadNbaData():
    with open('NBA_Data_Cleaned_df.pickle', 'rb') as file:
        nba_df = pickle.load(file)

    with open('teamID_mappings.pickle', 'rb') as file:
        team_mappings = pickle.load(file)
        teamID_newID_dict = team_mappings['teamID_newID_dict']
        
    nba_df[['team_id_home', 'team_id_away']] = nba_df[['team_id_home', 'team_id_away']].replace(teamID_newID_dict)

    X_df = nba_df.drop(columns=['season_id', 'new_season_id', 'wl_home', 'game_date', 'wl_away', 'fg_pct_home'])
    y_df = nba_df.loc[ : , 'wl_home' ]

    X_df.insert(10, 'rest_days_home', X_df.pop('rest_days_home'))
    X_df.insert(21, 'rest_days_away', X_df.pop('rest_days_away'))

    print(X_df.columns)

    X = X_df.to_numpy()
    y = y_df.to_numpy()

    return X,y

X,y = loadNbaData()

print("X shape: ", X.shape)
print("y shape: ", y.shape)

print("Home team: ", (X[:,0:10]).shape)
print("Away team: ", (X[:,11:21]).shape)
print("Season type: ", (X[:,22:]).shape)

Index(['team_id_home', 'fga_home', 'fg3a_home', 'oreb_home', 'dreb_home',
       'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pf_home',
       'rest_days_home', 'team_id_away', 'fga_away', 'fg3a_away', 'oreb_away',
       'dreb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away', 'pf_away',
       'rest_days_away', 'season_type'],
      dtype='object')
X shape:  (44549, 23)
y shape:  (44549,)
Home team:  (44549, 10)
Away team:  (44549, 10)
Season type:  (44549, 1)


In [23]:
import torch 

X_torch = torch.from_numpy(X)
y_torch = torch.from_numpy(y)

In [24]:
from torch.utils.data import Dataset, DataLoader

class NBADataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, idx):
        return self.X_data[idx], self.y_data[idx]

In [25]:
from torch.utils.data import random_split

dataset = NBADataset(X_torch, y_torch)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

batch_size = 128

train_loader = DataLoader(train_dataset, batch_size=128)
test_loader = DataLoader(test_dataset, batch_size=128)

## Create FFN module ##

In [ ]:
import torch.nn as nn

class FFN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FFN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out